In [2]:
import open3d as o3d
import time, builtins, tempfile, datetime, os 
from BetaPose import utils, chemtools, representations; 

import pytraj as pt 
import numpy as np 
from scipy.stats import entropy 
from scipy.ndimage import gaussian_filter 
from scipy.spatial import distance_matrix 

from BetaPose import utils, cluster

FEATURIZER_PARMS = {
  # Mask of components 
  "MASK_INTEREST" : ":LIG,MDL", 
  "MASK_ENVIRONMENT" : ":1-221",
  
  # POCKET SETTINGS
  "VOXEL_DIMENSION" : [12, 12, 12],    # Unit: 1 (Number of lattice in one dimension)
  "CUBOID_LENGTH" : [8,8,8],           # Unit: Angstorm (Need scaling)
  
  # SEARCH SETTINGS
  "UPDATE_INTERVAL" : 1, 
  "CUTOFF": 18, 
}

# Example workflow

from BetaPose import trajloader, data_io
from BetaPose import features, featurizer


# Load multiple trajectories
trajs = "/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_001_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_002_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_003_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_004_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_005_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_006_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_007_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_008_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_009_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_traj.nc%"
# trajs = "/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_001_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_002_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_003_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_004_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_005_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_006_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_007_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_009_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_010_traj.nc%"
trajs = trajs.strip("%").split("%")
tops = ["/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_END.pdb"] * 10
# tops = ["/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_END.pdb"] * 10
tloader = trajloader.TrajectoryLoader(trajs, tops);
trajectories = trajloader.TrajectoryLoader(trajs, tops); 

for traj in trajectories: 
  # Complete the trajectory information
  traj.strip(":T3P")
  print(traj.traj)
  #### traj.addcharge(); 
  
  # Initialize the featurizer since different trajectory might have distinct parameters
  # featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  feature_mass = features.MassFeature(); 
  # NOTE: in this step, the feature hooks back the feature and could access the featurizer by feat.featurer
  
  featurizer.register_feature(feature_mass)   
  
  repr_traji, fpfh_traji, features_traji = featurizer.run(traj, range(1,50,10), traj.top.select("@CA&:50-70"))
  
#   print(repr_traji.tolist())
  
  print(repr_traji.shape)
  print(fpfh_traji.shape)
  print(features_traji.shape)
#   featurizer.dump("repr_form", repr_traji, "/tmp/test.h5"); 
#   featurizer.dump("FPFH", np.array([(0,d) for d in fpfh_traji], dtype=object), "/tmp/test.h5"); 
  break
  
import pickle 
thedict = {
  "repr":repr_traji, 
  "fpfh":fpfh_traji, 
  "features":features_traji
}

with open('/tmp/test.pkl', 'wb') as f:
  # Write the object to the file
  pickle.dump(thedict, f)

pytraj.Trajectory, 1001 frames: 
Size: 0.056316 (GB)
<Topology: 2517 atoms, 188 residues, 38 mols, PBC with box type = cubic>
           
Center [0 0 0]
Found the MSMS executable msms_i86_64Linux2_2.6.1/msms.x86_64Linux2.2.6.1
23-05-04T11:33:40   : Frame 1: Generated 21 centers
23-05-04T11:33:49   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-04T11:33:49   : Frame 11: Generated 21 centers
23-05-04T11:33:57   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-04T11:33:57   : Frame 21: Generated 21 centers
23-05-04T11:34:03   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-04T11:34:03   : Frame 31: Generated 21 centers
23-05-04T11:34:10   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-04T11:34:10   : Frame 41: Generated 21 centers
23-05-04T11:34:17   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
(105, 72)
(105, 33, 600)
(105, 1, 12, 12, 12)


# Viewpoint Feature histogram


In [ ]:
import numpy as np

def compute_vfh(points, normals, viewpoint, bins=45):
  # Compute the centroid
  centroid = np.mean(points, axis=0); 
  # Compute the direction from the centroid to each point
  directions = viewpoint - centroid; 
  # Compute the angles between the directions and the normals
  angles = np.arccos(np.sum(normals * directions, axis=1) / np.linalg.norm(directions, axis=1))
  # Compute the viewpoint component histogram
  vfh_hist_viewpoint, _ = np.histogram(angles, bins=bins, range=(0, np.pi))

  # Compute the Extended FPFH component for each point
  extended_fpfh = np.zeros((len(points), bins))

  for i, (point, normal) in enumerate(zip(points, normals)):
      k_neighbors = np.arange(len(points))
      k_neighbors = k_neighbors[k_neighbors != i]
      extended_fpfh[i] = compute_pfh(points, normals, i, k_neighbors, bins)

  # Compute the Extended FPFH histogram by summing up the histograms for all points
  vfh_hist_extended = np.sum(extended_fpfh, axis=0)

  # Normalize the histograms
  vfh_hist_viewpoint = vfh_hist_viewpoint / np.sum(vfh_hist_viewpoint)
  vfh_hist_extended = vfh_hist_extended / np.sum(vfh_hist_extended)

  # Concatenate the viewpoint and extended histograms to form the VFH
  vfh = np.concatenate((vfh_hist_viewpoint, vfh_hist_extended))

  return vfh


In [3]:
import numpy as np

def compute_vfh_viewpoint_component(point_cloud, viewpoint, num_bins=45):
    # Compute the relative position of points in the point cloud
    relative_positions = point_cloud - viewpoint

    # Compute the distances from the viewpoint to each point in the point cloud
    distances = np.linalg.norm(relative_positions, axis=1)

    # Compute the relative angles (alpha, phi, theta) between the reference point and the points in the point cloud
    alpha = np.arctan2(relative_positions[:, 1], relative_positions[:, 0])
    phi = np.arctan2(relative_positions[:, 2], np.sqrt(relative_positions[:, 0] ** 2 + relative_positions[:, 1] ** 2))
    theta = np.arccos(relative_positions[:, 2] / distances)

    # Compute the histogram bin indices for alpha, phi, and theta
    alpha_indices = np.floor(alpha / (2 * np.pi) * num_bins).astype(int) % num_bins
    phi_indices = np.floor(phi / (np.pi) * num_bins).astype(int) % num_bins
    theta_indices = np.floor(theta / (np.pi) * num_bins).astype(int) % num_bins

    # Compute the viewpoint component of the VFH fingerprint
    vfh_viewpoint_component = np.zeros((num_bins, num_bins, num_bins))
    for i, j, k in zip(alpha_indices, phi_indices, theta_indices):
        vfh_viewpoint_component[i, j, k] += 1

    # Normalize the VFH viewpoint component
    vfh_viewpoint_component /= vfh_viewpoint_component.sum()

    return vfh_viewpoint_component

# Example usage
point_cloud = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
viewpoint = np.array([0, 0, 0])
vfh_viewpoint_component = compute_vfh_viewpoint_component(point_cloud, viewpoint)
print(vfh_viewpoint_component)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [5]:
import numpy as np 
import open3d as o3d
import time
from BetaPose import utils

from matplotlib.cm import inferno

def calc_tm(roll, pitch, yaw, translate=[0, 0, 0]):
    # Precompute trigonometric functions
    cos_roll, sin_roll = np.cos(roll), np.sin(roll)
    cos_pitch, sin_pitch = np.cos(pitch), np.sin(pitch)
    cos_yaw, sin_yaw = np.cos(yaw), np.sin(yaw)

    # Generate rotation matrices
    Rx = np.array([[1, 0, 0], [0, cos_roll, -sin_roll], [0, sin_roll, cos_roll]])
    Ry = np.array([[cos_pitch, 0, sin_pitch], [0, 1, 0], [-sin_pitch, 0, cos_pitch]])
    Rz = np.array([[cos_yaw, -sin_yaw, 0], [sin_yaw, cos_yaw, 0], [0, 0, 1]])

    # Combine rotations
    R = Rx @ Ry @ Rz

    # Create the final transformation matrix
    H = np.eye(4); 
    H[:3, :3] = R; 
    H[:3, 3] = np.array(translate).ravel()

    return H


def transform_pcd(pcd, trans_mtx): 
  # Homogenize the point cloud (add a row of ones)
  homogeneous_pcd = np.hstack((pcd, np.ones((pcd.shape[0], 1))));
  # Apply the transformation matrix to the point cloud
  transformed_pcd = np.dot(homogeneous_pcd, trans_mtx.T);
  # Remove the homogeneous coordinate (last column)
  transformed_pcd = transformed_pcd[:, :3];
  return transformed_pcd
  

pcd = np.arange(102*3).reshape(-1,3)
# pcd = np.asarray([np.zeros(102),np.zeros(102), np.arange(102)]).T
print(pcd)
pcd_out = np.array(pcd)

t1 = time.perf_counter()

for r in np.linspace(0, np.pi*2, 20): 
  for p in np.linspace(0, np.pi*2, 20): 
    for z in np.linspace(0, np.pi*2, 20): 
      TransMatrix = utils.calc_tm(r, p , z)
      pcd2 = utils.transform_pcd(pcd, TransMatrix)
      pcd_out = np.vstack((pcd_out, pcd2))
  
  
print("Method1: ", time.perf_counter() - t1,)
# print(pcd2)


# import matplotlib.pyplot as plt 

point_cloud = o3d.geometry.PointCloud()

# Set the points of the point cloud object
point_cloud.points = o3d.utility.Vector3dVector(pcd_out)

thecmap = inferno(np.linspace(0,1,len(pcd_out)))[:, :3]
print(thecmap)

point_cloud.colors = o3d.utility.Vector3dVector(thecmap)


# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])




[[  0   1   2]
 [  3   4   5]
 [  6   7   8]
 ...
 [297 298 299]
 [300 301 302]
 [303 304 305]]
Method1:  6.0915879950043745
[[1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 ...
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]]


In [10]:
pcd = np.arange(102*3).reshape(-1,3)
np.vstack((pcd,pcd))

array([[  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8],
       [  9,  10,  11],
       [ 12,  13,  14],
       [ 15,  16,  17],
       [ 18,  19,  20],
       [ 21,  22,  23],
       [ 24,  25,  26],
       [ 27,  28,  29],
       [ 30,  31,  32],
       [ 33,  34,  35],
       [ 36,  37,  38],
       [ 39,  40,  41],
       [ 42,  43,  44],
       [ 45,  46,  47],
       [ 48,  49,  50],
       [ 51,  52,  53],
       [ 54,  55,  56],
       [ 57,  58,  59],
       [ 60,  61,  62],
       [ 63,  64,  65],
       [ 66,  67,  68],
       [ 69,  70,  71],
       [ 72,  73,  74],
       [ 75,  76,  77],
       [ 78,  79,  80],
       [ 81,  82,  83],
       [ 84,  85,  86],
       [ 87,  88,  89],
       [ 90,  91,  92],
       [ 93,  94,  95],
       [ 96,  97,  98],
       [ 99, 100, 101],
       [102, 103, 104],
       [105, 106, 107],
       [108, 109, 110],
       [111, 112, 113],
       [114, 115, 116],
       [117, 118, 119],
       [120, 121, 122],
       [123, 124

In [66]:
import numpy as np

# Create a random point cloud (10 points, 3D)
point_cloud = np.random.rand(10, 3)

# Define a transformation matrix (4x4) - this example applies a rotation and a translation
transformation_matrix = np.array([
    [1, 0, 0, 1],  # Rotation around x-axis and translation along x-axis
    [0, np.cos(np.pi/4), -np.sin(np.pi/4), 2],  # Rotation around y-axis and translation along y-axis
    [0, np.sin(np.pi/4), np.cos(np.pi/4), 3],  # Rotation around z-axis and translation along z-axis
    [0, 0, 0, 1],  # Homogeneous coordinates
])

transformation_matrix = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

# Homogenize the point cloud (add a row of ones)
homogeneous_point_cloud = np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1))))

print("HOMO", homogeneous_point_cloud)

# Apply the transformation matrix to the point cloud
transformed_point_cloud = np.dot(homogeneous_point_cloud, transformation_matrix.T)

# Remove the homogeneous coordinate (last column)
transformed_point_cloud = transformed_point_cloud[:, :3]

print("Original Point Cloud:")
print(point_cloud)
print("\nTransformed Point Cloud:")
print(transformed_point_cloud)


HOMO [[0.07558461 0.51635333 0.53196232 1.        ]
 [0.30807784 0.48062533 0.82053643 1.        ]
 [0.12494007 0.39270109 0.34991867 1.        ]
 ...
 [0.00682978 0.54734804 0.06782893 1.        ]
 [0.94273368 0.35409856 0.41438046 1.        ]
 [0.97162929 0.65258015 0.12234071 1.        ]]
Original Point Cloud:
[[0.07558461 0.51635333 0.53196232]
 [0.30807784 0.48062533 0.82053643]
 [0.12494007 0.39270109 0.34991867]
 ...
 [0.00682978 0.54734804 0.06782893]
 [0.94273368 0.35409856 0.41438046]
 [0.97162929 0.65258015 0.12234071]]

Transformed Point Cloud:
[[0.07558461 0.51635333 0.53196232]
 [0.30807784 0.48062533 0.82053643]
 [0.12494007 0.39270109 0.34991867]
 ...
 [0.00682978 0.54734804 0.06782893]
 [0.94273368 0.35409856 0.41438046]
 [0.97162929 0.65258015 0.12234071]]
